# Postprocessing of upper and bottom lines
Fix the problem when the bottom line disappear and fix the upper line

In [100]:
import cv2
from pathlib import Path
import numpy as np
import pandas as pd

Import video

In [101]:
video_number = "3"
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
video_path = (
    project_root
    / "data"
    / f"recording_{video_number}"
    / f"Recording_{video_number}.mp4"
)
video_path = str(video_path)

# Load the video
cap = cv2.VideoCapture(video_path)

# Check
print(
    f"Opened: {cap.isOpened()}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, Total Frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}"
)

Opened: True, FPS: 59.94005994005994, Total Frames: 227.0


Import points

In [102]:
# Define the path to the CSV file
input_data_path = (
    project_root
    / "notebook"
    / "lane_detection"
    / "intermediate_data"
    / "lane_points"
    / f"lane_points_raw_{video_number}.csv"
)

# Load the CSV file into a DataFrame
points_df = pd.read_csv(input_data_path)

Function to determine when the bottom line disappear

In [103]:
"""if both are above return them
if one is above and the other is below return the one that is above and the other one 
    compute as the intersection of the lateral line with a line with teh same slope of the first frame that pass through the correct point
if both are below return None, None -> fix the prolem later
"""


def is_disappeared(bl_0, br_0, bl, br, tr, tl, max_y, threshold=0.99):
    if (
        bl[1] < max_y * threshold and br[1] < max_y * threshold
    ):  # both points are above the threshold -> keep them
        return bl, br
    return None, None

Compute the intersection point

In [104]:
def get_intersection(p1, p2, p3, p4):
    # Calculate the direction vectors of the lines
    dx1, dy1 = p2[0] - p1[0], p2[1] - p1[1]
    dx2, dy2 = p4[0] - p3[0], p4[1] - p3[1]

    # Calculate the determinant to check if lines are parallel
    determinant = dx1 * dy2 - dy1 * dx2
    if determinant == 0:
        return None  # Lines are parallel

    # Calculate the intersection point using the parametric equations of the lines
    t = ((p3[0] - p1[0]) * dy2 - (p3[1] - p1[1]) * dx2) / determinant
    intersection = (p1[0] + t * dx1, p1[1] + t * dy1)

    return intersection

Postprocessing upper line

In [105]:
def postprocessing_upper(points_df, bottom_y_distances):
    df = points_df[["Frame", "up_left_y"]].copy()
    df_length = len(df)
    # print('last 100 df:', df['up_left_y'].values[-100:])

    distances = np.diff(df["up_left_y"].values)
    # print('length:', len(distances), 'length df:', len(df))

    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    print("Mean distance:", mean_distance)
    print("Std distance:", std_distance)

    threshold = std_distance
    num_outliers = 100

    while num_outliers > 0:
        filtered_distances = np.where(np.abs(distances) > threshold, np.nan, distances)
        # print('Indexes with NaN in filtered_distances:', np.where(np.isnan(filtered_distances))[0])
        num_outliers = np.sum(np.isnan(filtered_distances))

        # # remove from df the rows with nan in filtered_distances
        # df = df.iloc[np.where(~np.isnan(filtered_distances))[0]]

        # Trova gli indici dei NaN
        nan_indices = np.where(np.isnan(filtered_distances))[0]
        # Calcola gli indici delle righe successive (i+1), evitando out-of-bounds
        next_indices = nan_indices + 1
        next_indices = next_indices[
            next_indices < len(df)
        ]  # assicurati di restare nel range
        # Rimuovi solo le righe successive
        df = df.drop(index=df.index[next_indices]).reset_index(drop=True)

        # compute angain the distances
        distances = np.diff(df["up_left_y"].values)
        # print('last 100 distances:', distances[-100:])

    # interpolate to found the missing values
    df["up_left_y"] = df["up_left_y"].interpolate(
        method="linear", limit_direction="both"
    )
    # print('last 100 df:', df['up_left_y'].values[-100:])
    # print('length:', len(df))

    # fill the remaining values at the end of the df with estimated values
    for i in range(len(df), df_length):
        if i >= len(df):
            df = pd.concat(
                [
                    df,
                    pd.DataFrame(
                        {
                            "Frame": [i],
                            "up_left_y": [
                                df.loc[i - 1, "up_left_y"] + bottom_y_distances[i - 1]
                            ],
                        }
                    ),
                ],
                ignore_index=True,
            )
    print("last 100 df:", df["up_left_y"].values[-100:])
    print("length:", len(df))

    for i, row in df.iterrows():
        y = row["up_left_y"]
        left_intersection = get_intersection(
            (points_df.loc[i, "bottom_left_x"], points_df.loc[i, "bottom_left_y"]),
            (points_df.loc[i, "up_left_x"], points_df.loc[i, "up_left_y"]),
            (0, y),
            (1000, y),
        )
        right_intersection = get_intersection(
            (points_df.loc[i, "bottom_right_x"], points_df.loc[i, "bottom_right_y"]),
            (points_df.loc[i, "up_right_x"], points_df.loc[i, "up_right_y"]),
            (0, y),
            (1000, y),
        )
        if left_intersection is not None and right_intersection is not None:
            points_df.loc[i, "up_left_x"] = left_intersection[0]
            points_df.loc[i, "up_left_y"] = left_intersection[1]
            points_df.loc[i, "up_right_x"] = right_intersection[0]
            points_df.loc[i, "up_right_y"] = right_intersection[1]
    return points_df

Main Algorithm

In [106]:
def postprocessing(points_df, heigth):
    # initial_points = points_df.iloc[0]
    bottom_disappeared = False
    frame = 10
    # compute the relative positions of points in the first frame
    # left_relative_position = (points_df.iloc[frame]["up_left_x"]-points_df.iloc[frame]["bottom_left_x"], points_df.iloc[frame]["up_left_y"]-points_df.iloc[frame]["bottom_left_y"])
    # right_relative_position = (points_df.iloc[frame]["up_right_x"]-points_df.iloc[frame]["bottom_right_x"], points_df.iloc[frame]["up_right_y"]-points_df.iloc[frame]["bottom_right_y"])

    df_copy = points_df.copy()
    bottom_y_distances = np.diff(df_copy["bottom_left_y"].values)

    for i in range(1, len(points_df)):
        bl_prev = (
            df_copy.iloc[i - 1]["bottom_left_x"],
            df_copy.iloc[i - 1]["bottom_left_y"],
        )
        br_prev = (
            df_copy.iloc[i - 1]["bottom_right_x"],
            df_copy.iloc[i - 1]["bottom_right_y"],
        )
        tl_prev = (df_copy.iloc[i - 1]["up_left_x"], df_copy.iloc[i - 1]["up_left_y"])
        tr_prev = (df_copy.iloc[i - 1]["up_right_x"], df_copy.iloc[i - 1]["up_right_y"])
        bl = (points_df.iloc[i]["bottom_left_x"], points_df.iloc[i]["bottom_left_y"])
        br = (points_df.iloc[i]["bottom_right_x"], points_df.iloc[i]["bottom_right_y"])
        tr = (points_df.iloc[i]["up_right_x"], points_df.iloc[i]["up_right_y"])
        tl = (points_df.iloc[i]["up_left_x"], points_df.iloc[i]["up_left_y"])

        if not bottom_disappeared:
            # compute relative positions
            window_size = 7
            if i < window_size:
                left_relative_position = (
                    tl_prev[0] - bl_prev[0],
                    tl_prev[1] - bl_prev[1],
                )
                right_relative_position = (
                    tr_prev[0] - br_prev[0],
                    tr_prev[1] - br_prev[1],
                )
            else:
                left_relative_positions = [
                    (
                        df_copy.iloc[j]["up_left_x"] - df_copy.iloc[j]["bottom_left_x"],
                        df_copy.iloc[j]["up_left_y"] - df_copy.iloc[j]["bottom_left_y"],
                    )
                    for j in range(i - window_size + 1, i + 1)
                ]
                right_relative_positions = [
                    (
                        df_copy.iloc[j]["up_right_x"]
                        - df_copy.iloc[j]["bottom_right_x"],
                        df_copy.iloc[j]["up_right_y"]
                        - df_copy.iloc[j]["bottom_right_y"],
                    )
                    for j in range(i - window_size + 1, i + 1)
                ]

                # # mean
                # left_relative_position = (sum(pos[0] for pos in left_relative_positions) / window_size,
                #                           sum(pos[1] for pos in left_relative_positions) / window_size)
                # right_relative_position = (sum(pos[0] for pos in right_relative_positions) / window_size,
                #                            sum(pos[1] for pos in right_relative_positions) / window_size)

                # lower
                left_relative_position = sorted(
                    left_relative_positions, key=lambda pos: pos[1], reverse=True
                )[1]
                right_relative_position = sorted(
                    right_relative_positions, key=lambda pos: pos[1], reverse=True
                )[1]
            # compute the new position of the bottom points in the current frame () if needed
            bl_new, br_new = is_disappeared(bl_prev, br_prev, bl, br, tr, tl, heigth)

            if bl_new is None and br_new is None:
                bottom_disappeared = True
                index_disappeared = i
                print(f"Bottom points disappeared at frame {i}.")
            else:  # consider correct the bottom points
                tr_mid = (
                    br_new[0] + right_relative_position[0],
                    br_new[1] + right_relative_position[1],
                )
                tl_mid = (
                    bl_new[0] + left_relative_position[0],
                    bl_new[1] + left_relative_position[1],
                )
                # Calculate the intersection point
                tr_new = get_intersection(tr_mid, tl_mid, br_new, tr)
                tl_new = get_intersection(tl_mid, tr_mid, bl_new, tl)

        if bottom_disappeared:  # consider correct the top poits
            bl_new = (
                tl[0] - left_relative_position[0],
                tl[1] - left_relative_position[1],
            )
            br_new = (
                tr[0] - right_relative_position[0],
                tr[1] - right_relative_position[1],
            )
            tr_new = tr
            tl_new = tl

        # Update the DataFrame with the new points
        points_df.at[i, "bottom_left_x"] = bl_new[0]
        points_df.at[i, "bottom_left_y"] = bl_new[1]
        points_df.at[i, "bottom_right_x"] = br_new[0]
        points_df.at[i, "bottom_right_y"] = br_new[1]
        points_df.at[i, "up_left_x"] = tl_new[0]
        points_df.at[i, "up_left_y"] = tl_new[1]
        points_df.at[i, "up_right_x"] = tr_new[0]
        points_df.at[i, "up_right_y"] = tr_new[1]
    df = postprocessing_upper(points_df, bottom_y_distances)
    if bottom_disappeared:
        for i in range(index_disappeared, len(points_df)):
            # compute the bottom points in the rows that are changed between df and points_df with relative positions
            bl_new = (
                df.iloc[i]["up_left_x"] - left_relative_position[0],
                df.iloc[i]["up_left_y"] - left_relative_position[1],
            )
            br_new = (
                df.iloc[i]["up_right_x"] - right_relative_position[0],
                df.iloc[i]["up_right_y"] - right_relative_position[1],
            )

            df.at[i, "bottom_left_x"] = bl_new[0]
            df.at[i, "bottom_left_y"] = bl_new[1]
            df.at[i, "bottom_right_x"] = br_new[0]
            df.at[i, "bottom_right_y"] = br_new[1]

    return df

Auxiliary functions

In [107]:
def lines_from_points(df):
    lines = []
    for i in range(len(df)):
        bottom = [
            df.iloc[i]["bottom_right_x"],
            df.iloc[i]["bottom_right_y"],
            df.iloc[i]["bottom_left_x"],
            df.iloc[i]["bottom_left_y"],
        ]
        top = [
            df.iloc[i]["up_right_x"],
            df.iloc[i]["up_right_y"],
            df.iloc[i]["up_left_x"],
            df.iloc[i]["up_left_y"],
        ]
        left = [
            df.iloc[i]["bottom_left_x"],
            df.iloc[i]["bottom_left_y"],
            df.iloc[i]["up_left_x"],
            df.iloc[i]["up_left_y"],
        ]
        right = [
            df.iloc[i]["bottom_right_x"],
            df.iloc[i]["bottom_right_y"],
            df.iloc[i]["up_right_x"],
            df.iloc[i]["up_right_y"],
        ]

        # Append the line to the list
        lines.append([bottom, top, left, right])
    return lines

In [108]:
"""Disegna la linea sul frame"""


def write_line_on_frame(frame, line):
    # Create a copy of the original frame to draw the first line
    modified_frame = np.copy(frame)

    # Extract the first line's rho and theta
    if line is not None:
        x1, y1, x2, y2 = line

        # Draw the first line on the frame
        cv2.line(modified_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # return the modified frame
    return modified_frame

In [109]:
def write_lines_on_frame(frame, lines):
    for i in range(len(lines)):
        # print('linea', i, ':', lines[i])
        frame = write_line_on_frame(frame, lines[i])
    return frame

Generate the video

In [110]:
# Reset the video to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Define the codec and create a VideoWriter object to save the modified frames
output_path = (
    project_root / "data" / f"recording_{video_number}" / "test_video" / "Lane_1.mp4"
)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use 'mp4v' codec for MP4 format
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))

# Process the lines
processed_points_df = postprocessing(points_df, frame_height)
lines = lines_from_points(processed_points_df)
# print('Processed lines:', lines)


# Loop through each frame in the video
frame_index = 0
while frame_index < len(lines):
    ret, video_frame = cap.read()
    if not ret:
        print("End of video or failed to read the frame at iteration", frame_index)
        break
    # print(f"Processing frame {frame_index}")

    # draw the lines on the frame
    modified_frame = write_lines_on_frame(video_frame, lines[frame_index])

    # Write the modified frame to the output video
    out.write(modified_frame)

    # Increment the frame index
    frame_index += 1

# Release the video capture and writer objects
# cap.release()
out.release()

print(f"Adjusted video saved to {output_path}")

Bottom points disappeared at frame 110.


C:\Users\miche\AppData\Local\Temp\ipykernel_4716\2827118193.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1244.9496606334842' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  points_df.at[i, "up_left_x"] = tl_new[0]
C:\Users\miche\AppData\Local\Temp\ipykernel_4716\2827118193.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '445.01300904977376' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  points_df.at[i, "up_left_y"] = tl_new[1]
C:\Users\miche\AppData\Local\Temp\ipykernel_4716\2827118193.py:74: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1536.9095394736842' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  poi

Mean distance: -1.1017699115044248
Std distance: 39.52051537789164
last 100 df: [459. 459. 459. 459. 460. 460. 460. 462. 460. 460. 460. 461. 461. 459.
 462. 464. 462. 464. 463. 467. 471. 474. 477. 482. 485. 486. 486. 492.
 493. 496. 495. 493. 494. 494. 501. 504. 504. 504. 500. 504. 505. 505.
 508. 505. 504. 506. 507. 508. 508. 508. 509. 508. 510. 510. 510. 511.
 506. 507. 509. 509. 509. 510. 510. 510. 512. 515. 513. 513. 513. 513.
 513. 513. 513. 513. 513. 513. 513. 513. 513. 513. 513. 513. 513. 513.
 513. 513. 513. 513. 513. 513. 513. 513. 513. 513. 513. 513. 513. 513.
 513. 513.]
length: 227


C:\Users\miche\AppData\Local\Temp\ipykernel_4716\2827118193.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1041.3792544570501' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, "bottom_right_x"] = br_new[0]
C:\Users\miche\AppData\Local\Temp\ipykernel_4716\2827118193.py:83: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '446.375' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, "bottom_left_x"] = bl_new[0]


Adjusted video saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\data\recording_3\test_video\Lane_1.mp4
